In [ ]:
!pip install -q torch==2.1.0 --index-url https://download.pytorch.org/whl/cu121

In [ ]:
!pip install -q "transformers==4.34.0" "datasets==2.13.0" "peft==0.6.2" "accelerate==0.21.0" "bitsandbytes==0.41.2.post2" "trl==0.4.7" "safetensors>=0.3.1" "gdown==4.6"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 96.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.0 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model = "mistralai/Mistral-7B-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model,
    quantization_config=bnb_config,
    device_map="cuda:0",
    trust_remote_code=True
)
model.config.use_cache = False

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
!gdown --id 1zfjg_3viiQ1IOR4AbsKiT4QCY1I1qWg8

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1zfjg_3viiQ1IOR4AbsKiT4QCY1I1qWg8
To: /content/Pandas_Code_SFT_Dataset.csv
100% 77.8k/77.8k [00:00<00:00, 95.3MB/s]


In [ ]:
import pandas as pd
from datasets import Dataset

dataset_df = pd.read_csv('Pandas_Code_SFT_Dataset.csv')
df = dataset_df.sample(frac=1).reset_index(drop=True)
print(df.columns)

Index(['Dataset Name', 'Columns with Sample Data', 'Question',
       'Expected Code'],
      dtype='object')


In [ ]:
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['Dataset Name', 'Columns with Sample Data', 'Question', 'Expected Code'],
    num_rows: 52
})

In [ ]:
def apply_prompt_template(examples):
  instruction = examples['Question']
  context = examples["Columns with Sample Data"]
  response = examples['Expected Code']

  full_prompt = f"""Below is an instruction that describes a task. Write a Python function using Pandas to accomplish the task described below.

### Instruction:
{instruction}

header columns with sample data:
{context}

### Response:
{response}

### End
"""
  return { "text": full_prompt }

dataset = dataset.map(apply_prompt_template)
print(dataset["text"][0])

Map:   0%|          | 0/52 [00:00<?, ? examples/s]

Below is an instruction that describes a task. Write a Python function using Pandas to accomplish the task described below.

### Instruction:
Which product had the highest total expenditure by customer FA101, and how much was it?

header columns with sample data:
data = {
    "invoice_date": ["2023-09-01", "2023-09-02", "2023-09-03", "2023-09-04", "2023-09-05"],
    "customer_code": ["C001", "C005", "C001", "C003", "C004"],
    "customer_name": ["Alice", "Bob", "Alice", "Charlie", "David"],
    "gender": ["Male", "Female", "Male", "Female", "Male"],
    "item_code": ["I001", "I002", "I003", "I004", "I005"],
    "item_name": ["Widget A", "Widget B", "Widget C", "Widget D", "Widget E"],
    "unit_price": [10, 20, 30, 40, 50],
    "quantity": [1, 2, 3, 1, 5],
    "total_amount": [10, 40, 90, 40, 250],
    "tax": [1, 4, 9, 4, 25],
    "currency": ["USD", "USD", "USD", "USD", "USD"],
    "discount": [0, 0, 0.05, 0, 0.1],
    "total_discount_after": [10, 40, 85.5, 40, 225]
}

### Response:
`

In [ ]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['q_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj', 'k_proj', 'v_proj'] # Choose all linear layers from the model
)

In [ ]:
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='CAUSAL_LM', inference_mode=False, r=64, target_modules={'up_proj', 'o_proj', 'gate_proj', 'v_proj', 'down_proj', 'k_proj', 'q_proj'}, lora_alpha=16, lora_dropout=0.1, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={})

In [ ]:
from transformers import TrainingArguments


training_params = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    optim="paged_adamw_32bit",
    save_steps=10,
    logging_steps=1,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=50,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [ ]:
training_params

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=True,
half_precision_backend=auto,
hub_always_push=False,
hub_model_

In [ ]:
from trl import SFTTrainer


trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_params,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:136: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/52 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,0.687100
2,0.642900
3,0.614300
4,0.382000
5,0.555100
6,0.405300
7,0.379600
8,0.178300
9,0.267300
10,0.356900


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

TrainOutput(global_step=50, training_loss=0.1869448335468769, metrics={'train_runtime': 577.9434, 'train_samples_per_second': 0.346, 'train_steps_per_second': 0.087, 'total_flos': 6267603584286720.0, 'train_loss': 0.1869448335468769, 'epoch': 3.85})

In [ ]:
from transformers import pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

In [ ]:
prompt = """
Below is an instruction that describes a task. Write a Python function using Pandas to accomplish the task described below.

### Instruction:
how many unique items are there

header columns with sample data:
data = {
    "invoice_date": ["2023-09-11", "2023-09-12", "2023-09-13", "2023-09-14", "2023-09-15"],
    "customer_code": ["C006", "C007", "C006", "C008", "C009"],
    "customer_name": ["Frank", "Grace", "Frank", "Hannah", "Ian"],
    "gender": ["Male", "Female", "Male", "Female", "Male"],
    "item_code": ["I005", "I004", "I003", "I002", "I001"],
    "item_name": ["Widget E", "Widget D", "Widget C", "Widget B", "Widget A"],
    "unit_price": ["$50", "$40", "$30", "$20", "$10"],
    "quantity": [2, 3, 1, 4, 5],
    "total_amount": [100, 120, 30, 80, 50],
    "tax": [10, 12, 3, 8, 5],
    "currency": ["USD", "USD", "USD", "EUR", "EUR"],
    "discount": [0.05, 0, 0, 0.1, 0.05],
    "total_discount_after": [95, 120, 30, 72, 47.5]
}

### Response:
"""


result = pipe(prompt, max_length=550)
generated_text = result[0]['generated_text']
print(generated_text.split("### End")[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Below is an instruction that describes a task. Write a Python function using Pandas to accomplish the task described below.

### Instruction:
how many unique items are there

header columns with sample data:
data = {{
    "invoice_date": ["2023-09-11", "2023-09-12", "2023-09-13", "2023-09-14", "2023-09-15"],
    "customer_code": ["C006", "C007", "C006", "C008", "C009"],
    "customer_name": ["Frank", "Grace", "Frank", "Hannah", "Ian"],
    "gender": ["Male", "Female", "Male", "Female", "Male"],
    "item_code": ["I005", "I004", "I003", "I002", "I001"],
    "item_name": ["Widget E", "Widget D", "Widget C", "Widget B", "Widget A"],
    "unit_price": ["$50", "$40", "$30", "$20", "$10"],
    "quantity": [2, 3, 1, 4, 5],
    "total_amount": [100, 120, 30, 80, 50],
    "tax": [10, 12, 3, 8, 5],
    "currency": ["USD", "USD", "USD", "EUR", "EUR"],
    "discount": [0.05, 0, 0, 0.1, 0.05],
    "total_discount_after": [95, 120, 30, 72, 47.5]
}}

### Response:
```
def process(df):
    # Count th

In [ ]:
from huggingface_hub import login
from google.colab import userdata

In [ ]:
login(token=userdata.get('niru_hf_write'))

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
trainer.save_model("Mistral_7B_SFT_Finetune_Pandas_Code_Gen_Final")

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
total_params_billion = total_params / 1e9
print(f"Total parameters: {total_params_billion:.2f} Billion")

Total parameters: 3.92 Billion


In [ ]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    "/content/Mistral_7B_SFT_Finetune_Pandas_Code_Gen_Final",
    low_cpu_mem_usage=True,
)

# Merge LoRA and base model
merged_model = model.merge_and_unload()

total_params = sum(p.numel() for p in merged_model.parameters())
total_params_billion = total_params / 1e9
print(f"Total parameters: {total_params_billion:.2f} Billion")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Total parameters: 7.24 Billion


In [ ]:
type(merged_model)

transformers.models.mistral.modeling_mistral.MistralForCausalLM

In [ ]:
# push merged model to the hub
merged_model.push_to_hub("Mistral_7B_SFT_Finetune_Pandas_Code_Gen_Final",private=True,safe_serialization=True)
tokenizer.push_to_hub("Mistral_7B_SFT_Finetune_Pandas_Code_Gen_Final",private=True)

model-00002-of-00003.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/9.08G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nirusanan/Mistral_7B_SFT_Finetune_Pandas_Code_Gen_Final/commit/aecc544f9b20096753d45e9e9e9e254cb8c478fa', commit_message='Upload tokenizer', commit_description='', oid='aecc544f9b20096753d45e9e9e9e254cb8c478fa', pr_url=None, pr_revision=None, pr_num=None)